<a href="https://colab.research.google.com/github/marcellinus-witarsah/speech-to-text-model/blob/main/speech-to-text-models/deep_speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>